# 9. Pythonic Object
- Python data모델은 사용자가 정의한 자료형도 내장 자료형과 같이 자연스럽게 동작할 수 있음
    - Duck Typing덕분에 특정 자료형을 상속하지 않고도 가능함
    - Object에 필요한 method만을 구현하면 기대한 대로 동작함
- What we are going to do in this chapter?
    - 실제 Python object와 동일하게 동작하는 user-defined class 만들기
- In this chapter,
    - repr(), bytes()등 object를 다른 방식으로 표현하는 내장 함수의 지원
    - class method로 대안 생성자 구현
    - format() 내장 함수와 str.format() method에서 사용하는 포맷 언어 확장
    - 읽기 전용 접근만 허용하는 속성 제공
    - 집합 및 dict key로 사용할 수 있도록 object를 hashable하게 만들기
    - __slots__ 를 사용하여 메모리 절약하기
    - @staticmethod, @classmathod
    - Python에서 비공개 및 보호된 속성: 사용법, 관례, 한계

## 9.1 Object 표현
- repr(): object를 개발자가 보고자 하는 형태로 표현한 문자열로 return
- str(): object를 사용자가 보고자 하는 형태로 표현한 문자열로 return
- repr(), str() method를 지원하려면 __repr()__, __str()__를 구현해야 함
- __bytes__(), __format__()
    - __bytes__(): __str__()와 비슷하지만, bytes() method에 의해 호출되어 object를 byte sequence로 표현함
    - __format__(): 내장함수 format()과 str.format()를 둘다 사용. 특별 format code를 사용해서 object를 표현하는 문자열 반환

## 9.2 Vector Class의 부활

In [42]:
from array import array
import math

class Vector2d_v0:
    typecode = 'd' #Vector2d와 bytes간의 변환에 사용하는 class 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    # bytes를 생성하기 위해 typecode를 bytes로 표현 & 객체를 반복해서 생성한 배열에서 변환된 bytes와 연결
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)) 
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [43]:
v1 = Vector2d_v0(3,4)
print('v1.x, v1.y: ', v1.x, v1.y)
x, y = v1 
print('x, y: ', x, y)
print('v1: ', v1)
v1_clone = eval(repr(v1))
print('v1 == v1_clone: ', v1 == v1_clone)
print('v1 is v1_clone: ', v1 is v1_clone)
octets = bytes(v1)
print('bytes(v1): ', octets)
print('abs(v1): ', abs(v1))
print('bool(v1): {}, bool(Vector2d_v0(0, 0)): {}'.format( bool(v1), bool(Vector2d_v0(0, 0))))



v1.x, v1.y:  3.0 4.0
x, y:  3.0 4.0
v1:  (3.0, 4.0)
v1 == v1_clone:  True
v1 is v1_clone:  False
bytes(v1):  b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
abs(v1):  5.0
bool(v1): True, bool(Vector2d_v0(0, 0)): False


## 9.4 @classmethod와 @staticmethod
- @Classmethod
    - object가 아닌 class에 연산을 수행하는 method
    - class자체를 첫 번째 parameter로 받게 만듦
    - 아래 예제에서 _randomize()_는 __init__()안의 instance variable을 전혀 사용하지 않음. 즉, instance method없이 바로 class에 적용 가능    

In [44]:
# Before
import random
class Randomize1:
    RANDOM_CHOICE = 'abcdefg'
    
    def __init__(self, char_num):
        self.char_num = char_num
    
    def _randomize(self, random_chars=3):
        return ''.join(random.choice(self.RANDOM_CHOICE) for _ in range(random_chars))

    
class Randomize2:
    RANDOM_CHOICE = 'abcdefg'
    
    def __init__(self, char_num):
        self.char_num = char_num
    
    @classmethod
    def _randomize(cls, random_chars=3):
        return ''.join(random.choice(cls.RANDOM_CHOICE) for _ in range(random_chars))

ran1 = Randomize1(5)
print(ran1._randomize())
print(Randomize2._randomize())

cca
egd


- @classmethod
    - 또 다른 예제
    - https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner

In [45]:
class Date(object):
    
    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year

    # input이 '10-09-2012' 형식으로 들어올 경우, 이 text를 '-'로 parsing해서 각각 day, month, year에 넣은 Date object 생성
    @classmethod
    def from_string(cls, date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        date1 = cls(day, month, year)
        return date1
    
    # classmethod나 object method처럼 첫번째 param에 cls나 self가 들어가지 않음
    @staticmethod
    def is_date_valid(date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        return day <= 31 and month <=12 and year  <= 3000
    
date2 = Date.from_string('11-09-2012')
is_date = Date.is_date_valid('11-09-2012')
print('{}/{}/{} is a date : {}'.format(date2.year, date2.month, date2.day, is_date))

2012/9/11 is a date : True


In [46]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args
    
print(Demo.klassmeth())
print(Demo.klassmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


## 9.5 Formatted print
- format()내장 함수와 str.format() method는 실제 format 작업을 __format__(format_spec) method에 위임함
    - format_spec: 포맷 명시자(format specifier)
        - format(my_obj, format_spec)의 2번째 인수
        - str.format()에 사용된 format string 안에 {}로 구분한 대체 필드 안에서의 콜론 뒤의 문자열

In [50]:
brl = 1/2.43
print(brl)
print(format(brl, '0.4f')) # 0,4f 가 format specifier(포맷 명시자))
print('1 BRL = {rate:.2f} USD'.format(rate=brl))

0.4115226337448559
0.4115
1 BRL = 0.41 USD


In [55]:
print(format(42, 'b'))
print(format(2/3, '.1%'))
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')
print("It's now {:%I:%M %p}".format(now))

101010
66.7%
It's now 05:56 PM


## 9.6 Hashable Vector2d
- <h4>\__hash__() method를 구현해야 Vector2d를 hashable하게 할 수 있음</h4>
    - 먼저, Vector2d object를 불변형으로 만들어야 함

- <h4>\__hash__()</h4>
    - int형을 반환해야 함
    - 동일하다고 판단되는 object는 동일한 hash값을 가져야 함
    - \__eq__() method가 사용하는 object의 attribute를 사용하여 hash값을 계산하는 것이 이상적임
        - component의 hash에 bit단위 XOR 연산자를 사용하는 것을 권장

In [55]:
class Vector2d:
    #__slots__ = ('__x', '__y')
    typecode = 'd'
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)
    
    @property # property decorator는 property의 getter method를 나타냄 
    def x(self): # 노출시키는 공개 속성명을 따라서 getter method의 이름을 지정함
        return self.__x # 단지, self.__x를 반환
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        cls_name = type(self).__name__
        return '{}({!r}, {!r})'.format(cls_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __format__(self, fmt_spec = ''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

### Various method tests

In [56]:
#2-dim vector class
v1= Vector2d(3,4)
print('v1.x, v1.y                            : {}, {}'.format(v1.x, v1.y))
x, y = v1
print('x, y                                     : {}, {}'.format(x, y))
print('v1                                       : {}'.format(v1))
v1_clone = eval(repr(v1))
print('v1 == v1_clone                    : {}'.format(v1 == v1_clone))
print('v1                                       : {}'.format(v1))
octets = bytes(v1)
print('octets                                 : {}'.format(octets))
print('abs(v1)                               : {}'.format(abs(v1)))
print('bool(v1), bool(Vector2d(0,0)): {}, {}'.format(bool(v1), bool(Vector2d(0,0))))

v1.x, v1.y                            : 3.0, 4.0
x, y                                     : 3.0, 4.0
v1                                       : (3.0, 4.0)
v1 == v1_clone                    : True
v1                                       : (3.0, 4.0)
octets                                 : b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
abs(v1)                               : 5.0
bool(v1), bool(Vector2d(0,0)): True, False


In [57]:
#frombytes()class method test
v1_clone = Vector2d.frombytes(bytes(v1))
print('v1_clone         : {}'.format(v1_clone))
print('v1 == v1_clone: {}'.format(v1 == v1_clone))

v1_clone         : (3.0, 4.0)
v1 == v1_clone: True


In [58]:
#format method test
print('format(v1)         : {}'.format(format(v1)))
print("format(v1, '.2f'): {}".format(format(v1, '.2f')))
print("format(v1, '.3e): {}".format(format(v1, '.3e')))

format(v1)         : (3.0, 4.0)
format(v1, '.2f'): (3.00, 4.00)
format(v1, '.3e): (3.000e+00, 4.000e+00)


In [59]:
# angle() method test
print("Vector2d(0,0).angle()                                  : {}".format(Vector2d(0,0).angle()))
print("Vector2d(0,0).angle()                                  : {}".format(Vector2d(1,0).angle()))
eps = 1e-7
print("abs(Vector2d(0,1).angle() - math.pi/2 < eps : {}".format(abs(Vector2d(0,1).angle() - math.pi/2 < eps)))
print("abs(Vector2d(1,1).angle() - math.pi/4 < eps : {}".format(abs(Vector2d(1,1).angle() - math.pi/4 < eps)))

Vector2d(0,0).angle()                                  : 0.0
Vector2d(0,0).angle()                                  : 0.0
abs(Vector2d(0,1).angle() - math.pi/2 < eps : 1
abs(Vector2d(1,1).angle() - math.pi/4 < eps : 1


In [60]:
# polar coordinate를 활용한 format() test
print("format(Vector2d(1,1), 'p')      : {}".format(format(Vector2d(1,1), 'p')))
print("format(Vector2d(1,1), '.3ep') : {}".format(format(Vector2d(1,1), '.3ep')))
print("format(Vector2d(1,1), '0.5fp'): {}".format(format(Vector2d(1,1), '0.5fp')))

format(Vector2d(1,1), 'p')      : <1.4142135623730951, 0.7853981633974483>
format(Vector2d(1,1), '.3ep') : <1.414e+00, 7.854e-01>
format(Vector2d(1,1), '0.5fp'): <1.41421, 0.78540>


In [61]:
# 'x', 'y' read-only property test
print(v1.x, v1.y)
v1.x = 123

3.0 4.0


AttributeError: can't set attribute

In [62]:
# hash test
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
print('hash(v1), hash(v2): {}, {}'.format(hash(v1), hash(v2)))
print('len(set([v1, v2]))   : {}'.format(len(set([v1, v2]))))

hash(v1), hash(v2): 7, 384307168202284039
len(set([v1, v2]))   : 2


## 9.7. Python의 private and protected attribute
- <h4>Subclass에서 'private'성격의 attribute를 변경하지 못하게 하는 메커니즘</h4>
    - Name mangling(이름 장식): attribute 명을 2개의 underline으로 시작하게 정의하면, python은 underline과 class name을 변수명 앞에 붙여 객체의 \__dict__ 에 저장함
    - 위의 Vector2d에서 요소(components)는 private attribute이며, Vector2d object는 불변형이다.

In [63]:
v1 = Vector2d(3,4)
print(v1.__dict__) # name mangling
print(v1._Vector2d__x)

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}
3.0


## 9.8 \__slots__ 클래스 속성으로 공간 절약하기
- <h4>Python은 object의 attributes들을 각 object 안의 \__dict__()라는 dictionary형 attribute에 저장함</h4>
- <h4>이 때, dictionary는 빠른 접근 속도를 제공하기 위해, 내부에 hash table을 유지함. --> 메모리 사용량 부담이 상당히 큼</h4>
    - 만약 속성이 몇개 없는 수백만 개의 object를 다룬다면, \__slots__() class attribute를 사용하여, 메모리 사용량을 크게 줄일 수 있음
    - \__slots__ attribute는 python이 object attribute를 dictionary 대신 tuple에 저장함
    - super class에서 상속받은 \__slots__ attribute는 subclass에 영향을 미치지 않음. 각 클래스에서 개별적으로 정의된 속성만 고려
- <h4>\__slots__를 정의하려면, 이 이름의 클래스 attribute를 생성하고 여기에 object attribute 식별자들을 담은 문자열의 반복형을 할당</h4>
    - 불변형인 tuple을 사용하면 \__slots__정의를 변경할 수 없음
- <h4>\__slots__를 class에 정의함으로써, '이 attributes들이 이 class object가 가지는 attributes'임을 알려줌</h4>
    - python interpereter가 이 attribute들을 각 object의 tuple형 구조체에 저장함 --> 각 object마다 \__dict__ attribute를 유지해야 하는 부담을 덜어내줌
- <h4>주의!</h4>
    - \__dict__ 를 slots에 추가하지 않는 한 objects는 slots에 나열된 attribute만 가질 수 있음
        - 그러나, \__dict__ 를 slots에 추가해버리면, 메모리 절감 효과가 반감될 수 있음
        - 그렇다고 class의 user가 object에 새로운 속성을 추가할 수 없게 하려고, slots을 사용하면 안된다.
        - slots의 목적은 메모리 사용을 최적화해서 user에게 도움을 주기 위한것이지, user를 억압하기 위한 것이 아님!
    - Interpreter는 상속된 slots attribute를 무시함 --> 각 클래스마다 slots attribute를 재정의해줘야 함
    - \__weakref__를 slots에 추가하지 않으면 object가 weak referecence의 대상이 될 수 있음

In [65]:
# 사용법
# class Vector2d:
    # __slots__ = ('__x', '__y')
    # typecode = 'd'